<h3>Tokenizer Evaluation on HAR, Sex 

Compare classification of magnitude model to tokenized magnitude with various quantisations. 

In [2]:
# Configure GPU

import sys

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)
cudnn.benchmark = True


ModuleNotFoundError: No module named 'torch'

Set up

In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Compiled with CUDA:", torch.version.cuda)
print("GPU support enabled:", torch.backends.cuda.is_built())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Load data

In [ ]:
import pandas as pd

#tokenized string for each participant - about a day of data. 
pid_tokens=pd.read_csv("pid_token_string.csv")

In [6]:
pid_tokens['pid'] = pid_tokens['pid'].str[:4]
pid_tokens.head()

,pid,token_string
0,P030,b b bb b bbb bb b b bb bbbbbcbbbbbbbbbbbbbbbc...
1,P036,kjfgghhjlhhhihhighggjigheddddddcdefefeekkgjjhi...
2,P113,bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbcbbbebbbbb...
3,P048,cbb gebbbb bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb...
4,P099,c cbbcbbbbbbcccccccccccccccccccccccccccccbbcc...


In [4]:
metadata= pd.read_csv("/data/UKBB/kyra/capture24/metadata.csv")

In [8]:
#combine meta data and token strings]
meta_tokens= pd.merge(metadata, pid_tokens, on='pid', how='inner')
meta_tokens.head()

,pid,age,sex,token_string
0,P001,38-52,F,bb bbbbbbbbb bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb...
1,P002,30-37,F,cccbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb...
2,P003,30-37,F,bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb...
3,P004,53+,F,b ebbbbbbbbbcbbbbbbbbbcbbbbbbbdgbbbbcbbbbbbbb...
4,P005,38-52,F,cccccccccccccccccccccccccccccccccccccccccccccc...


Train/ Test split

In [ ]:
# Hold out participants P101-P151 for testing (51 participants)
test_ids = [f'P{i}' for i in range(101,152)]
mask_test = np.isin(pid, test_ids)
mask_train = ~mask_test
X_train, Y_train, T_train, pid_train = \
    X[mask_train], Y[mask_train], T[mask_train], pid[mask_train]
X_test, Y_test, T_test, pid_test = \
    X[mask_test], Y[mask_test], T[mask_test], pid[mask_test]
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Transformer - with arbitarly long input sequence. -predict sex

In [ ]:
import torch
import torch.nn as nn

class TokenTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, nhead=4, num_layers=2, num_classes=2, max_len=2048):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        # x: (batch, seq_len)
        positions = torch.arange(x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        x = x.transpose(0, 1)  # Transformer expects (seq_len, batch, embed_dim)
        x = self.transformer(x)
        x = x.mean(dim=0)  # Global average pooling over sequence
        return self.fc(x)



In [ ]:
        Example usage:
        vocab_size = tokenizer.get_vocab_size()
        model = TokenTransformerClassifier(vocab_size)
        token_ids_batch = torch.tensor([[1,2,3,4,0,0], [5,6,7,0,0,0]])  # (batch, seq_len)
        logits = model(token_ids_batch)